Keras 는 scikit-learn 수준으로 추상화 되어 있는 패키지 입니다. Keras 의 간단한 모델 설정 함수들을 실행하면 그 안에서 tensorflow 의 여러 함수들이 실행됩니다. 즉 keras 는 backend 로 tensorflow 를 이용합니다. 그렇기 때문에 반드시 tensorflow 를 설치해야 합니다. 그리고 tensorflow 2.x 부터는 Keras 가 tensorflow 의 내부에 포함되어 있습니다. 이를 위해서 tensorflow 를 설치하세요. 이 때 tensorflow 가 이용하는 몇몇 패키지들이 다른 패키지들과 충돌을 일으킬 수 있기 때문에 가능하면 가상환경을 만드신 다음에 설치하시는걸 권장드립니다.

```
pip install upgrade tensorflow
```

In [1]:
import tensorflow
from tensorflow import keras

print(f'keras=={keras.__version__}')
print(f'tensorflow=={tensorflow.__version__}')

keras==2.4.0
tensorflow==2.3.0


00 의 Feed forward neural network 의 실습에 이용했던 fashion-MNIST 데이터를 이용합니다.

In [2]:
import sys
mnist_utils_path = '../../../../fashion-mnist/utils/'
mnist_data_path  = '../../../../fashion-mnist/data/fashion/'
sys.path.append(mnist_utils_path)

학습용 데이터와 테스트용 데이터를 모두 불러옵니다. `numpy.unique()` 를 이용하면 클래스 개수를 확인할 수 있습니다.

In [3]:
import numpy as np
from mnist_reader import load_mnist

train, train_labels = load_mnist(mnist_data_path, kind='train')
test, test_labels   = load_mnist(mnist_data_path, kind='t10k')

n_classes = np.unique(train_labels).shape[0]

print(train.shape)
print(train_labels.shape)
print(test.shape)
print(test_labels.shape)
print(n_classes)

(60000, 784)
(60000,)
(10000, 784)
(10000,)
10


이미지는 여러 개의 체널로 구성됩니다. 이미지의 크기를 (height, width, channel) 로 혹은 (channel, height, width) 로 정의할 수 있습니다. 지금 이용하는 tensorflow 의 형식을 확인합니다.

In [4]:
from keras import backend as K

K.image_data_format()

'channels_last'

이미지의 크기를 표현하는 방법에 따라 input shape 을 다르게 정의하며, (n data, channel, height, width) 혹은 (n data, height, width, channel) 로 input data 의 shape 을 변경합니다. 또한 이미지 데이터는 uint8 형식의 어레이입니다. 뉴럴 네트워크가 잘 학습할 수 있도록 [0, 1] 사이의 실수값으로 이를 변환합니다.

In [5]:
if K.image_data_format() == 'channels_first':
    train_X = train.reshape(train.shape[0], 1, 28, 28)
    test_X = test.reshape(test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    train_X = train.reshape(train.shape[0], 28, 28, 1)
    test_X = test.reshape(test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

print(f'range = ({train_X.min()}, {train_X.max()})')
print(f'dtype = {train_X.dtype}')
print(f'train_X shape = {train_X.shape}')
print(f'test_X  shape = {test_X.shape}')

train_X = train_X.astype(np.float32) / 255
test_X = test_X.astype(np.float32) / 255

print(f'\nrange = ({train_X.min()}, {train_X.max()})')
print(f'dtype = {train_X.dtype}')
print(f'train_X shape = {train_X.shape}')
print(f'test_X  shape = {test_X.shape}')

range = (0, 255)
dtype = uint8
train_X shape = (60000, 28, 28, 1)
test_X  shape = (10000, 28, 28, 1)

range = (0.0, 1.0)
dtype = float32
train_X shape = (60000, 28, 28, 1)
test_X  shape = (10000, 28, 28, 1)


`train_labels` 은 shape 이 (60000,) 인 column vector 입니다. 이를 one-hot representation 으로 변경합니다.

In [6]:
train_y = keras.utils.to_categorical(train_labels, n_classes)
test_y = keras.utils.to_categorical(test_labels, n_classes)

print(f'train_y shape = {train_y.shape}')
print(f'test_y  shape = {test_y.shape}', end='\n\n')

print(train_y[:3])

train_y shape = (60000, 10)
test_y  shape = (10000, 10)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


Keras 에서 feed forward 방식으로 모델을 구성할 때에는 `keras.models.Sequential` 을 이용할 수 있습니다. input 이 Sequential 의 처음부터 마지막까지 이동한다는 의미입니다.

Conv2D 는 2 차원의 kernel 을 적용하는 convolutional filter 를 의미합니다. filters, kernel_size 등의 패러매터값은 CNN 의 강의노트에 등장한 개념입니다. `kernel_initializer` 는 뉴럴 네트워크의 initializer 에서 다뤘던 방법입니다. 활성함수로 ReLU 를 이용할 때 좋은 성능을 보여준다고 알려져 있습니다.

Conv2D 가 적용된 이후 max pooling 을 수행하고, activation map 을 `Flatten()` 을 이용하여 flatten vector 로 만들 수 있습니다. 그 뒤, fully connected layer 를 추가하고 마지막에 softmax 를 활성함수로 지니는 10 차원의 fully connected layer 를 추가합니다.

Sequential 에 add 되는 각 레이어들은 앞선 레이어의 output shape 로부터 다음 레이어의 input shape 을 알 수 있습니다. 그러나 Sequential 의 첫번째 레이어는 input data 의 모양을 모르기 때문에 `input_shape` 을 반드시 지정해줘야 합니다.

이처럼 모델 구성이 모두 끝나면 `compile()` 을 수행합니다. 이 때 학습에 이용할 loss function 와 optimizer 를 설정합니다.

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

cnn = Sequential()
cnn.add(
    Conv2D(
        filters=32,
        kernel_size=(3,3),
        strides=(1,1),
        activation='relu',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        input_shape = input_shape
    ))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(units=128, activation='relu'))
cnn.add(Dense(64, activation='relu'))
cnn.add(Dense(n_classes, activation='softmax'))

cnn.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

`summary()` 함수를 실행하면 layers 의 구성이 표현됩니다. Output shape 의 첫칸의 None 은 batch size 가 다르게 들어올 수 있기 때문에 이를 가변적으로 열어둔 것입니다.

In [8]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               102528    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8

Keras 는 scikit learn 처럼 fit 함수와 predict 함수를 제공합니다. predict 함수에 test data 를 입력해봅니다. output 의 크기가 (batch, n classes) 이기 때문에 보기가 힘드니 이를 transpose 해서 확인해봅니다. 세 개의 테스트 데이터는 어떤 특정 클래스에 속하지 않고 대체로 비슷한 확률값을 지닙니다.

In [9]:
np.set_printoptions(precision=7, suppress=True)

print(f'shape of output = {cnn.predict(test_X[:3]).shape}')
print(cnn.predict(test_X[:3]).T)

shape of output = (3, 10)
[[0.0945626 0.0968767 0.0966252]
 [0.1057699 0.1145191 0.1188056]
 [0.1005651 0.1099868 0.1011962]
 [0.1106071 0.1093005 0.1085985]
 [0.0896339 0.0848579 0.0904503]
 [0.1073859 0.0975858 0.1015748]
 [0.0881518 0.0925867 0.0993383]
 [0.1023306 0.1131312 0.1021632]
 [0.0945683 0.0811743 0.0841571]
 [0.1064248 0.0999811 0.0970907]]


Compiled 된 모델의 fit 함수에 학습데이터를 입력합니다. 이 때 batch size 와 epochs 를 지정할 수 있습니다. 진행상황을 살펴보기 위하여 verbose 도 조절할 수 있습니다. validation data 를 입력하면 매 epoch 마다 이 데이터를 이용한 성능 평가를 수행합니다. 앞서 compile 함수에서 metrics 에 'accuracy' 를 입력하였기 때문에 정확도 기준에서 성능을 측정합니다.

파이썬의 time.time() 함수를 이용하면 현재 시각이 출력됩니다. 20 epochs 의 학습 시간도 측정해 봅니다.

In [10]:
from time import time


batch_size = 128
epochs = 20

train_time = time()

history = cnn.fit(
    train_X, train_y,
    batch_size = batch_size,
    epochs = epochs,
    verbose = 2, # 0: silent, 1: progress bar, 2: one line per epoch
    validation_data = (test_X, test_y)
)

train_time = time() - train_time

Epoch 1/20
469/469 - 8s - loss: 0.6028 - accuracy: 0.7774 - val_loss: 0.4379 - val_accuracy: 0.8380
Epoch 2/20
469/469 - 8s - loss: 0.3804 - accuracy: 0.8605 - val_loss: 0.3749 - val_accuracy: 0.8678
Epoch 3/20
469/469 - 8s - loss: 0.3275 - accuracy: 0.8791 - val_loss: 0.3403 - val_accuracy: 0.8787
Epoch 4/20
469/469 - 8s - loss: 0.3001 - accuracy: 0.8894 - val_loss: 0.3233 - val_accuracy: 0.8832
Epoch 5/20
469/469 - 8s - loss: 0.2729 - accuracy: 0.8997 - val_loss: 0.3027 - val_accuracy: 0.8887
Epoch 6/20
469/469 - 8s - loss: 0.2558 - accuracy: 0.9053 - val_loss: 0.2912 - val_accuracy: 0.8953
Epoch 7/20
469/469 - 8s - loss: 0.2405 - accuracy: 0.9101 - val_loss: 0.2799 - val_accuracy: 0.8984
Epoch 8/20
469/469 - 8s - loss: 0.2248 - accuracy: 0.9165 - val_loss: 0.2714 - val_accuracy: 0.9002
Epoch 9/20
469/469 - 8s - loss: 0.2102 - accuracy: 0.9211 - val_loss: 0.2737 - val_accuracy: 0.9009
Epoch 10/20
469/469 - 8s - loss: 0.2005 - accuracy: 0.9254 - val_loss: 0.2689 - val_accuracy: 0.9049

테스트 시간도 측정해봅니다.

In [11]:
test_time = time()
score = cnn.evaluate(test_X, test_y)
test_time = time() - test_time

313/313 [==============================] - 1s 2ms/step - loss: 0.3014 - accuracy: 0.9120


`Model.evaluate()` 함수의 결과는 앞서 verbose message 로 출력된 validation data 에 대한 loss 와 accuracy 입니다.

In [12]:
print(f'train time = {train_time:.5} sec')
print(f'test  time = {test_time:.5} sec')
print(f'evaluation score = {score}')

train time = 158.04 sec
test  time = 0.63726 sec
evaluation score = [0.3013738989830017, 0.9120000004768372]


실제로 패러매터가 학습되었는지 확인해봅니다. 이번에는 각 데이터들이 특정 클레스에 속할 확률값이 커졌습니다 (entropy 가 줄어들었습니다)

In [13]:
print(f'shape of output = {cnn.predict(test_X[:3]).shape}')
print(cnn.predict(test_X[:3]).T)

shape of output = (3, 10)
[[0.        0.0000001 0.       ]
 [0.        0.        1.       ]
 [0.        0.999931  0.       ]
 [0.        0.        0.       ]
 [0.        0.0000438 0.       ]
 [0.0000004 0.        0.       ]
 [0.        0.0000252 0.       ]
 [0.0000257 0.        0.       ]
 [0.        0.        0.       ]
 [0.9999738 0.        0.       ]]


Keras 의 패러매터들을 numpy 로 옮겨올 수도 있습니다. max pooling 은 패러매터가 없습니다. 그렇기 때문에 convolutional filter x2 와 fully connected x3 에 대한 패러매터를 가져와 크기를 확인해 봅니다.

In [14]:
for layer in cnn.layers:
    name = layer.name
    if name[:5] in ['conv2', 'dense']:        
        weights = layer.weights[0].numpy()
        bias = layer.bias.numpy()
        print(f'[{name}] weights: {weights.shape}, bias: {bias.shape}')

[conv2d] weights: (3, 3, 1, 32), bias: (32,)
[conv2d_1] weights: (3, 3, 32, 32), bias: (32,)
[dense] weights: (800, 128), bias: (128,)
[dense_1] weights: (128, 64), bias: (64,)
[dense_2] weights: (64, 10), bias: (10,)


각 hidden layers 의 output 도 가져올 수 있습니다. 이는 조금 복잡한데, 위 구문은 tensorflow 의 구조를 공부하신 다음에 그 의미를 살펴보시기 바랍니다. 지금은 이 함수를 그대로 가져다 이용하셔도 좋습니다. 모든 레이어의 값이 아니라 특정 레이어의 값만을 가져오고 싶다면 `func` 의 두번째 lsit 를 다르게 정의하면 됩니다.

(참고: https://keras.io/getting_started/faq/#how-can-i-obtain-the-output-of-an-intermediate-layer-feature-extraction)

```python
def get_hidden_vectors(model, inputs, mode=0):
    func = K.function(
        inputs = [model.layers[0].input],
        outputs = [model.layers[3].output]
    )
```

각 레이어의 이름과 hidden vectors 의 shape 을 함께 출력합니다. layer.name 은 summary() 함수의 name 입니다. _1, _2 와 같은 suffix 는 메모리 상에 만들어진 레이어들을 구분하기 위한 인덱스 입니다.

In [15]:
def get_hidden_vectors(model, inputs, mode=0):
    func = K.function(
        inputs = [model.layers[0].input],
        outputs = [layer.output for layer in model.layers]
    )
    hiddens = func(inputs)
    names = [layer.name for layer in model.layers]
    return hiddens, names

hiddens, names = get_hidden_vectors(cnn, test_X[:3])

print(f'num layers = {len(hiddens)}\n')
for h, name in zip(hiddens, names):
    print(f'[{name:16}]: {h.shape}')

print(type(h))

num layers = 8

[conv2d          ]: (3, 26, 26, 32)
[max_pooling2d   ]: (3, 13, 13, 32)
[conv2d_1        ]: (3, 11, 11, 32)
[max_pooling2d_1 ]: (3, 5, 5, 32)
[flatten         ]: (3, 800)
[dense           ]: (3, 128)
[dense_1         ]: (3, 64)
[dense_2         ]: (3, 10)
<class 'numpy.ndarray'>


`Model.fit()` 함수의 출력값인 history 에는 학습 과정에 대한 정보가 포함되어 있습니다. 이 안에는 history 라는 dict 가 포함되어 있습니다. verbose message 에 출력된 값들이 포함되어 있는데, 이를 bokeh 를 이용하여 lineplot 으로 그려봅니다.

In [16]:
import bokeh
from bokeh.palettes import Spectral4
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import SingleIntervalTicker

print(bokeh.__version__)
output_notebook()

# bokeh >= 1.4.0 부터 legend -> legend_label 로 이름이 바뀌었습니다.
def linechart(x, y, line_color='grey', line_width=3, legend_label=None, p=None, title=None):
    if p is None:
        p = figure(width=800, height=400, title=title)
        p.xaxis.axis_label = 'Epoch'
        p.xaxis.ticker = SingleIntervalTicker(interval=1)
        p.xaxis.major_label_standoff = 1
    p.line(x=x, y=y, line_color=line_color, line_width=line_width,
           legend_label=legend_label, alpha=0.8, line_dash=(4,4))
    return p

1.4.0


Loading BokehJS ...

In [17]:
x = np.arange(len(history.history['loss']))
p = None
title = 'Training performance of CNN'
for color, (legend, y) in zip(Spectral4, history.history.items()):
    p = linechart(x, y, line_color=color, legend_label=legend, p=p, title=title)
show(p)

앞서 scikit-learn 으로 연습한 feed-forward 모델도 keras 로 손쉽게 만들 수 있습니다. 대부분의 내용은 CNN 모델 구성과 같지만, 이번에는 `add()` 함수에 name 을 입력하였습니다. 인덱스로 구분하는 것이 복잡할 때에는 이름을 정의하여 구분이 쉽도록 만듭니다.

In [18]:
ff = Sequential()
ff.add(Dense(200, activation='relu', name='hidden_1', input_shape=(784,)))
ff.add(Dense(50, activation='relu', name='hidden_2'))
ff.add(Dense(n_classes, activation='softmax', name='output'))
ff.compile(
    loss = keras.losses.categorical_crossentropy,
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

history = ff.fit(
    train_X.reshape(train_X.shape[0],-1),
    train_y,
    batch_size = batch_size,
    epochs = epochs,
    verbose = 2, # 0: silent, 1: progress bar, 2: one line per epoch
    validation_data = (test_X.reshape(test_X.shape[0],-1), test_y)
)

Epoch 1/20
469/469 - 1s - loss: 0.5214 - accuracy: 0.8174 - val_loss: 0.4208 - val_accuracy: 0.8520
Epoch 2/20
469/469 - 1s - loss: 0.3836 - accuracy: 0.8640 - val_loss: 0.3955 - val_accuracy: 0.8586
Epoch 3/20
469/469 - 1s - loss: 0.3396 - accuracy: 0.8773 - val_loss: 0.3665 - val_accuracy: 0.8661
Epoch 4/20
469/469 - 1s - loss: 0.3162 - accuracy: 0.8844 - val_loss: 0.3706 - val_accuracy: 0.8655
Epoch 5/20
469/469 - 1s - loss: 0.2957 - accuracy: 0.8911 - val_loss: 0.3466 - val_accuracy: 0.8767
Epoch 6/20
469/469 - 1s - loss: 0.2819 - accuracy: 0.8960 - val_loss: 0.3573 - val_accuracy: 0.8674
Epoch 7/20
469/469 - 1s - loss: 0.2691 - accuracy: 0.9001 - val_loss: 0.3719 - val_accuracy: 0.8632
Epoch 8/20
469/469 - 1s - loss: 0.2577 - accuracy: 0.9021 - val_loss: 0.3446 - val_accuracy: 0.8741
Epoch 9/20
469/469 - 1s - loss: 0.2476 - accuracy: 0.9085 - val_loss: 0.3344 - val_accuracy: 0.8811
Epoch 10/20
469/469 - 1s - loss: 0.2399 - accuracy: 0.9096 - val_loss: 0.3253 - val_accuracy: 0.8864

MNIST, Fashion-MNIST 모두 어려운 데이터는 아니기 때문에 어느 정도의 성능은 보여줍니다. 하지만 CNN 이 조금 더 좋은 성능을 보여줍니다.

In [19]:
x = np.arange(len(history.history['loss']))
p = None
title = 'Training performance of Feed-forward h=(200,50)'
for color, (legend, y) in zip(Spectral4, history.history.items()):
    p = linechart(x, y, line_color=color, legend_label=legend, p=p, title=title)
show(p)

앞서 정의한 이름으로 `summary()` 의 결과값이 출력됩니다.

In [20]:
ff.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_1 (Dense)             (None, 200)               157000    
_________________________________________________________________
hidden_2 (Dense)             (None, 50)                10050     
_________________________________________________________________
output (Dense)               (None, 10)                510       
Total params: 167,560
Trainable params: 167,560
Non-trainable params: 0
_________________________________________________________________


In [21]:
for layer in ff.layers:
    name = layer.name
    weights = layer.weights[0].numpy()
    bias = layer.bias.numpy()
    print(f'[{name:9}] weights: {weights.shape}, bias: {bias.shape}')

[hidden_1 ] weights: (784, 200), bias: (200,)
[hidden_2 ] weights: (200, 50), bias: (50,)
[output   ] weights: (50, 10), bias: (10,)


앞서 정의한 `get_hidden_vectors()` 함수를 재활용 할 수 있습니다. feed forward 모델의 hidden vectors 를 가져옵니다. 이름을 지정하니 보기가 정말 편리합니다.

In [22]:
hiddens, names = get_hidden_vectors(ff, test_X[:3].reshape(3,-1))

print(f'num layers = {len(hiddens)}\n')
for h, name in zip(hiddens, names):
    print(f'[{name:8}]: {h.shape}')

num layers = 3

[hidden_1]: (3, 200)
[hidden_2]: (3, 50)
[output  ]: (3, 10)


학습한 모델을 저장할 수 있습니다. `Model.save()` 함수를 이용합니다.

In [23]:
ff.save('keras_ff.h5')

학습된 모델을 읽어올 수 있습니다. `test_X` 에 대하여 evaluation 을 수행하니 동일한 성능을 보여줍니다.

In [24]:
from keras.models import load_model

ff_loaded = load_model('keras_ff.h5')
ff_loaded.evaluate(
    test_X.reshape(test_X.shape[0],-1), test_y,
    verbose = 0
)

[0.3475989103317261, 0.8895000219345093]